In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime


# Getting to Know the Data

1. We pull 20,000 rows from the database using an API to understand the structure of the data and its contents.
2. From the values in the `ofns_desc` column, we create a list.
3. We create a loop that sends a series of queries through the API for each value in the list.
4. We save the results in a CSV file.


In [2]:
# API Connection
from sodapy import Socrata

client = Socrata("data.cityofnewyork.us", "yUnYRTjXHWdK6ATxbXOsJR52v", timeout=60)

# First 20000 results, returned as JSON from API / converted to Python list of dictionaries by sodapy.
results = client.get("qgea-i56i", limit=20000)
df = pd.DataFrame.from_records(results)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   cmplnt_num                   20000 non-null  object
 1   cmplnt_fr_dt                 20000 non-null  object
 2   cmplnt_fr_tm                 20000 non-null  object
 3   cmplnt_to_dt                 18586 non-null  object
 4   cmplnt_to_tm                 20000 non-null  object
 5   addr_pct_cd                  19996 non-null  object
 6   rpt_dt                       20000 non-null  object
 7   ky_cd                        20000 non-null  object
 8   ofns_desc                    20000 non-null  object
 9   pd_cd                        19982 non-null  object
 10  pd_desc                      20000 non-null  object
 11  crm_atpt_cptd_cd             20000 non-null  object
 12  law_cat_cd                   20000 non-null  object
 13  boro_nm                      20

In [4]:
# show values from column ofns_desc
df['ofns_desc'].value_counts()

ofns_desc
PETIT LARCENY                           4229
HARRASSMENT 2                           2992
ASSAULT 3 & RELATED OFFENSES            2078
GRAND LARCENY                           1991
CRIMINAL MISCHIEF & RELATED OF          1636
FELONY ASSAULT                          1028
VEHICLE AND TRAFFIC LAWS                 689
GRAND LARCENY OF MOTOR VEHICLE           668
ROBBERY                                  636
BURGLARY                                 620
MISCELLANEOUS PENAL LAW                  601
OFF. AGNST PUB ORD SENSBLTY &            598
DANGEROUS DRUGS                          341
SEX CRIMES                               289
OFFENSES AGAINST PUBLIC ADMINI           227
DANGEROUS WEAPONS                        221
INTOXICATED & IMPAIRED DRIVING           164
FORGERY                                  153
THEFT-FRAUD                              114
OFFENSES INVOLVING FRAUD                  94
FRAUDS                                    77
CRIMINAL TRESPASS                         71


In [5]:
# define list of values from column ofns_desc and make a new list
ofns_desc = df['ofns_desc'].value_counts().index.tolist()
ofns_desc

['PETIT LARCENY',
 'HARRASSMENT 2',
 'ASSAULT 3 & RELATED OFFENSES',
 'GRAND LARCENY',
 'CRIMINAL MISCHIEF & RELATED OF',
 'FELONY ASSAULT',
 'VEHICLE AND TRAFFIC LAWS',
 'GRAND LARCENY OF MOTOR VEHICLE',
 'ROBBERY',
 'BURGLARY',
 'MISCELLANEOUS PENAL LAW',
 'OFF. AGNST PUB ORD SENSBLTY &',
 'DANGEROUS DRUGS',
 'SEX CRIMES',
 'OFFENSES AGAINST PUBLIC ADMINI',
 'DANGEROUS WEAPONS',
 'INTOXICATED & IMPAIRED DRIVING',
 'FORGERY',
 'THEFT-FRAUD',
 'OFFENSES INVOLVING FRAUD',
 'FRAUDS',
 'CRIMINAL TRESPASS',
 'POSSESSION OF STOLEN PROPERTY',
 'RAPE',
 'UNAUTHORIZED USE OF A VEHICLE',
 'ADMINISTRATIVE CODE',
 'OFFENSES AGAINST THE PERSON',
 'NYS LAWS-UNCLASSIFIED FELONY',
 'OTHER STATE LAWS (NON PENAL LA',
 'ARSON',
 'OTHER OFFENSES RELATED TO THEF',
 'MURDER & NON-NEGL. MANSLAUGHTER',
 "BURGLAR'S TOOLS",
 'OFFENSES AGAINST PUBLIC SAFETY',
 'FRAUDULENT ACCOSTING',
 'OTHER STATE LAWS',
 'GAMBLING',
 'THEFT OF SERVICES',
 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED',
 'CANNABIS RELATED OFFENSES',
 '

In [8]:
# First 500000 results, returned as JSON from API / converted to Python list of dictionaries by sodapy. 
# Make a loop and put through api query for every value fron ofns_desc list. 
# Include columns cmplnt_fr_dt	cmplnt_fr_tm and only one value from the ofns_desc list 

ofns_desc_list = df['ofns_desc'].value_counts().index.tolist()

# Set today's date
today = datetime.now().strftime("%Y-%m-%d")

# Create an empty list to store the results
results_list = []

# Loop through each value in the ofns_desc list
for ofns_desc_value in ofns_desc_list:
    # Escapovanje apostrofa u vrednosti
    ofns_desc_value_escaped = ofns_desc_value.replace("'", "''")
    
    # Formiranje upita sa escapovanim apostrofom
    query = f"ofns_desc='{ofns_desc_value_escaped}' AND cmplnt_fr_dt >= '2010-01-01' AND cmplnt_fr_dt <= '{today}'"
   
    # Query the API for the specific value from ofns_desc list
    results = client.get("qgea-i56i",
                         select="cmplnt_fr_dt, cmplnt_fr_tm, ofns_desc, boro_nm, susp_age_group, susp_race, susp_sex, vic_age_group, vic_race, vic_sex",
                         where=query,
                         limit=10000000)
    
    # Convert the results to a DataFrame
    results_df = pd.DataFrame.from_records(results)
    
    # Append the results to the results_list
    results_list.append(results_df)

# Concatenate all the DataFrames in the results_list
final_df = pd.concat(results_list)




In [9]:
final_df.info()



<class 'pandas.core.frame.DataFrame'>
Index: 6230872 entries, 0 to 125
Data columns (total 10 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   cmplnt_fr_dt    object
 1   cmplnt_fr_tm    object
 2   ofns_desc       object
 3   boro_nm         object
 4   susp_age_group  object
 5   susp_race       object
 6   susp_sex        object
 7   vic_age_group   object
 8   vic_race        object
 9   vic_sex         object
dtypes: object(10)
memory usage: 522.9+ MB


In [10]:
# Show how many unique values from ofns_desc column
final_df['ofns_desc'].value_counts()

ofns_desc
PETIT LARCENY                           1120973
HARRASSMENT 2                            848787
ASSAULT 3 & RELATED OFFENSES             673484
CRIMINAL MISCHIEF & RELATED OF           616129
GRAND LARCENY                            555028
DANGEROUS DRUGS                          291715
OFF. AGNST PUB ORD SENSBLTY &            288325
FELONY ASSAULT                           268562
ROBBERY                                  212569
BURGLARY                                 198312
MISCELLANEOUS PENAL LAW                  172588
DANGEROUS WEAPONS                        123301
OFFENSES AGAINST PUBLIC ADMINI           113012
GRAND LARCENY OF MOTOR VEHICLE           105912
VEHICLE AND TRAFFIC LAWS                  90754
SEX CRIMES                                78270
INTOXICATED & IMPAIRED DRIVING            70272
FORGERY                                   60602
THEFT-FRAUD                               56831
CRIMINAL TRESPASS                         54920
FRAUDS                        

In [11]:
# Reset index column
final_df.reset_index(drop=True, inplace=True)


In [12]:
display(final_df.head())
display(final_df.tail())

,cmplnt_fr_dt,cmplnt_fr_tm,ofns_desc,boro_nm,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex
0,2010-10-27T00:00:00.000,19:30:00,PETIT LARCENY,BRONX,(null),(null),(null),45-64,UNKNOWN,F
1,2021-05-24T00:00:00.000,03:00:00,PETIT LARCENY,BRONX,UNKNOWN,UNKNOWN,U,45-64,WHITE,M
2,2010-02-10T00:00:00.000,12:00:00,PETIT LARCENY,STATEN ISLAND,(null),(null),(null),25-44,WHITE,M
3,2010-02-09T00:00:00.000,15:00:00,PETIT LARCENY,STATEN ISLAND,(null),(null),(null),45-64,UNKNOWN,M
4,2010-07-29T00:00:00.000,08:00:00,PETIT LARCENY,STATEN ISLAND,(null),(null),(null),65+,WHITE,F


,cmplnt_fr_dt,cmplnt_fr_tm,ofns_desc,boro_nm,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex
6230867,2022-09-22T00:00:00.000,16:07:00,ESCAPE 3,QUEENS,25-44,BLACK,M,UNKNOWN,UNKNOWN,E
6230868,2022-08-07T00:00:00.000,00:51:00,ESCAPE 3,BROOKLYN,25-44,BLACK,M,UNKNOWN,UNKNOWN,E
6230869,2022-12-19T00:00:00.000,06:36:00,ESCAPE 3,QUEENS,25-44,WHITE HISPANIC,F,UNKNOWN,UNKNOWN,D
6230870,2022-11-26T00:00:00.000,14:15:00,ESCAPE 3,QUEENS,<18,BLACK,M,<18,BLACK,M
6230871,2022-05-25T00:00:00.000,22:15:00,ESCAPE 3,QUEENS,<18,BLACK,M,UNKNOWN,UNKNOWN,E


In [16]:
# Save the final_df to a CSV file
final_df.to_csv('../Resources/final_df.csv', index=False)


# Next step

1. Findings are strongly supported with numbers and visualizations
